In [1]:
import pandas as pd
import urllib.request

In [2]:
# week numbers for every holiday season (nov-jan) from season 2016-17 to 2019-20
# separated for better visibility

# szn_2016_17 = [161105,161112,161119,161126,161203,161210,161217,161224,
#                161231,170107,170114,170121,170128]
             
# szn_2017_18 = [171104,171111,171118,171125,171202,171209,171216,171223,
#                171230,180106,180113,180120,180127]
             
# szn_2018_19 = [181103,181110,181117,181124,181201,181208,181215,181222,
#                181229,190105,190112,190119,190126]

szn_2019_20 = [191102,191109,191116,191123,191130,191207,191214,191221,
              191228,200104,200111,200118,200125]

szn_2019_20 = []

# each season added into one list 
week_id = szn_2019_20

In [3]:
# concatenating all weeks into one dataframe
def import_data(week_id):
    df_list = []
    for week_num in week_id:
        df_list.append(pd.read_csv(f"http://web.mta.info/developers/data/nyct/turnstile/turnstile_{week_num}.txt"))

    return pd.concat(df_list)
mta_df = import_data(week_id)

In [4]:
mta_df

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,00:00:00,REGULAR,7247322,2455491
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,04:00:00,REGULAR,7247336,2455499
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,08:00:00,REGULAR,7247351,2455532
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,12:00:00,REGULAR,7247463,2455623
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,16:00:00,REGULAR,7247755,2455679
...,...,...,...,...,...,...,...,...,...,...,...
205096,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,01/24/2020,04:00:00,REGULAR,5554,420
205097,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,01/24/2020,08:00:00,REGULAR,5554,420
205098,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,01/24/2020,12:00:00,REGULAR,5554,420
205099,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,01/24/2020,16:00:00,REGULAR,5554,420


In [5]:
from sqlalchemy import create_engine

In [6]:
engine = create_engine("sqlite:///mta.db", echo=True)

sqlite_connection = engine.connect()

2021-03-30 17:56:17,057 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-03-30 17:56:17,058 INFO sqlalchemy.engine.base.Engine ()
2021-03-30 17:56:17,059 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-03-30 17:56:17,059 INFO sqlalchemy.engine.base.Engine ()


In [7]:
engine.table_names()

2021-03-30 17:56:17,806 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-03-30 17:56:17,806 INFO sqlalchemy.engine.base.Engine ()


[]

In [8]:
# importing the df as a sqlite table in mta.db
sqlite_table = "MTA_2019_20"
mta_df.to_sql(sqlite_table, sqlite_connection, if_exists='fail')

2021-03-30 17:56:19,528 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("MTA_2019_20")
2021-03-30 17:56:19,529 INFO sqlalchemy.engine.base.Engine ()
2021-03-30 17:56:19,530 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("MTA_2019_20")
2021-03-30 17:56:19,530 INFO sqlalchemy.engine.base.Engine ()
2021-03-30 17:56:19,531 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "MTA_2019_20" (
	"index" BIGINT, 
	"C/A" TEXT, 
	"UNIT" TEXT, 
	"SCP" TEXT, 
	"STATION" TEXT, 
	"LINENAME" TEXT, 
	"DIVISION" TEXT, 
	"DATE" TEXT, 
	"TIME" TEXT, 
	"DESC" TEXT, 
	"ENTRIES" BIGINT, 
	"EXITS                                                               " BIGINT
)


2021-03-30 17:56:19,532 INFO sqlalchemy.engine.base.Engine ()
2021-03-30 17:56:19,534 INFO sqlalchemy.engine.base.Engine COMMIT
2021-03-30 17:56:19,534 INFO sqlalchemy.engine.base.Engine CREATE INDEX "ix_MTA_2019_20_index" ON "MTA_2019_20" ("index")
2021-03-30 17:56:19,535 INFO sqlalchemy.engine.base.Engine ()
2021-03-30 17:56

In [9]:
sqlite_connection.close()

In [10]:
engine = create_engine("sqlite:///mta.db")

In [11]:
engine.table_names()

['MTA_2019_20']

In [13]:
# checking first 10 rows of MTA_2017_20 table in mta.db

pd.read_sql('SELECT * FROM MTA_2019_20 LIMIT 10;', engine)

,index,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,0,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,00:00:00,REGULAR,7247322,2455491
1,1,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,04:00:00,REGULAR,7247336,2455499
2,2,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,08:00:00,REGULAR,7247351,2455532
3,3,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,12:00:00,REGULAR,7247463,2455623
4,4,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,16:00:00,REGULAR,7247755,2455679
5,5,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,20:00:00,REGULAR,7248159,2455733
6,6,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/27/2019,00:00:00,REGULAR,7248310,2455757
7,7,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/27/2019,04:00:00,REGULAR,7248339,2455765
8,8,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/27/2019,08:00:00,REGULAR,7248356,2455782
9,9,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/27/2019,12:00:00,REGULAR,7248418,2455860
